## Psyco Examples

In [6]:
import os

import psycopg2
import shapefile  # pyshp
from shapely.geometry import Point, MultiPoint, Polygon
from shapely.wkb import loads

## Import Point Shapefile to Spatial Database

In [4]:
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = 'pwd' #os.environ["PG_PWD"]

ddl = """
DROP TABLE IF EXISTS public.nyc_museums;

CREATE TABLE public.nyc_museums
(
    id SERIAL NOT NULL 
        PRIMARY KEY, 
    NAME VARCHAR(255) NULL,
    URL VARCHAR(255) NULL,
    --location    GEOMETRY(POLYGON, 4326)   NULL
    location    GEOGRAPHY(POINT, 4326)   NULL
);

CREATE INDEX nyc_museums_points_gix 
ON public.nyc_museums 
USING GIST (location);
"""
point_cloud = []

with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c: #, shapefile.Reader(nyc_path) as nyc_museums:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(ddl)
        c.commit()

        # Execute DML
        cursor = c.cursor()

        dml = """
        INSERT INTO public.nyc_museums (NAME, URL, location)
        VALUES (%s, %s, ST_GeomFromText(%s, 4326))
        """

        # Read both attributes and geometries.
#         for srec in nyc_museums.shapeRecords():
#             p = Point(srec.shape.points[0][0], srec.shape.points[0][1])
#             point_cloud.append(p)
#             cursor.execute(dml, (srec.record['NAME'], srec.record['URL'], p.wkt))

        p = Point(31.8457, 102.3676)
        cursor.execute(dml, ('met', 'www.met.com', p.wkt))
        c.commit()
        
    except Exception as e:
        c.rollback()
        raise e

print("Completed.")
# mpt = MultiPoint(point_cloud)
# mpt

Completed.


## Read Points from Spatial Database

In [5]:
dml = """
SELECT id
		, name
		, url
		, location 
FROM	public.nyc_museums
LIMIT 10;
"""

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c:
    cursor = c.cursor()

    cursor.execute(dml)
    
    for rec in cursor.fetchall():
        pt = loads(rec[3], hex=True)
        print(pt)

POINT (31.8457 77.6324)


## Import Polygons to Spatial Database

In [ ]:
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

ddl = """
DROP TABLE IF EXISTS public.us_states;

CREATE TABLE public.us_states
(
    id SERIAL NOT NULL 
        PRIMARY KEY, 
    NAME VARCHAR(100) NULL,
    CODE CHAR(2) NULL,
    ABBREV CHAR(2) NULL,
    AREA BIGINT NULL,
    -- NAD 83
    -- TODO: Can be MultiPolygon. How to handle both?
    location    GEOMETRY(POLYGON, 4269)   NULL
);

CREATE INDEX us_states_poly_gix 
ON public.us_states
USING GIST (location);
"""
shp_path = os.path.join(os.environ["DATA_DIR"], 'us_states', 'tl_2014_us_state.shp')

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c, \
    shapefile.Reader(shp_path) as shp:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(ddl)
        c.commit()

        # Execute DML
        cursor = c.cursor()

        dml = """
        INSERT INTO public.us_states (NAME, CODE, ABBREV, AREA, location)
        VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 4269))
        """

        # Read both attributes and geometries.
        for srec in shp.shapeRecords()[:5]:
            if srec.record['STUSPS'] not in ('AK', 'HI'):
                poly = Polygon(srec.shape.points)
                cursor.execute(dml, (srec.record['NAME'], srec.record['STATEFP'], 
                                     srec.record['STUSPS'], srec.record['ALAND'], poly.wkt))

        c.commit()
        
    except Exception as e:
        c.rollback()
        raise e

## Load from CSV

In [ ]:
branch_file = "data/patient_branch_zip.csv"
ddl = """
DROP TABLE IF EXISTS public.patient_branch;

CREATE TABLE public.patient_branch
(
    id                 SERIAL NOT NULL 
        PRIMARY KEY, 
    patient_key        VARCHAR(32) NOT NULL,
    patient_latitude   FLOAT NOT NULL,
    patient_longitude  FLOAT NOT NULL,
    branch_key         INT NOT NULL,
    branch_latitude    FLOAT NOT NULL,
    branch_longitude   FLOAT NOT NULL,
    -- TODO: Use 3857 (Web Mercator)
    --location    GEOMETRY(POLYGON, 4326)   NULL
    patient_location   GEOGRAPHY(POINT, 4326)   NOT NULL,
    branch_location    GEOGRAPHY(POINT, 4326)   NOT NULL
);

CREATE INDEX patient_points_gix 
ON public.patient_branch 
USING GIST (patient_location);

CREATE INDEX branch_points_gix 
ON public.patient_branch 
USING GIST (branch_location);
"""

In [ ]:
%%time
with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c, open(branch_file) as csvfile:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(ddl)
        c.commit()

        # Execute DML
        cursor = c.cursor()

        dml = """
        INSERT INTO public.patient_branch (
            patient_key,
            patient_latitude,
            patient_longitude,
            branch_key,
            branch_latitude,
            branch_longitude,
            patient_location,
            branch_location
        )
        VALUES (%s, %s, %s, %s, %s, %s, 
                ST_GeomFromText(%s, 4326), ST_GeomFromText(%s, 4326))
        """

        # Read both attributes and geometries.
        n = 0
        reader = csv.DictReader(csvfile)
        for row in reader:
            n += 1
            
            if not row['PatientLatitude'] or not row['PatientLongitude']:
                continue
                
            p1 = Point(float(row['PatientLatitude']), float(row['PatientLongitude']))

            try:
            
                if not row['BranchLatitude'] or not row['BranchLongitude']:
                    continue

                p2 = Point(float(row['BranchLatitude']), float(row['BranchLongitude']))
            except ValueError as e:
                print(row)
                
            cursor.execute(dml, (
                row['PatientKey'], 
                row['PatientLatitude'], 
                row['PatientLongitude'], 
                int(float(row['BranchKey'])), 
                row['BranchLatitude'], 
                row['BranchLongitude'], 
                p1.wkt,
                p2.wkt
            ))
        c.commit()
        print(f"Inserted {n} records")
        
    except Exception as e:
        c.rollback()
        raise e